In [68]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load Tokenized Data

In [69]:
tr = pd.read_csv("../data/tokenized_train.csv")
ts = pd.read_csv("../data/tokenized_test.csv")

In [70]:
def preprocess(tokens):
    tokens = eval(tokens)
    tokens = list(filter(lambda x: x!='', tokens))
    tokens = list(map(lambda x: x.lower(), tokens))
    return tokens

In [71]:
tr["tokenizedReview"] = tr["tokenizedReview"].apply(lambda x: preprocess(x))
ts["tokenizedReview"] = ts["tokenizedReview"].apply(lambda x: preprocess(x))

In [72]:
tr.head()

,review,label,tokenizedReview
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,"[ร้าน, อาหาร, ใหญ่, มา, กกกกกกก, \n, เลี้ยว, เ..."
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,"[อาหาร, ที่, นี่, เป็น, อาหาร, จีน, แคะ, ที่, ..."
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,"[ปอเปี๊ยะสด, ทุก, วัน, นี้, รู้สึก, ว่า, หากิน..."
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,"[รัาน, คัพเค้ก, ใน, เมือง, ไทย, มี, ไม่, มาก, ..."
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,"[อร่อย, !, !, !, เดิน, ผ่าน, digital, gateway,..."


In [73]:
ts.head()

,reviewID,review,tokenizedReview
0,1,ร้านนี้จะอยู่เส้นสันกำแพง-แม่ออน เลยแยกบ่...,"[ร้าน, นี้, จะ, อยู่, เส้น, สัน, กำแพง, -, แม่..."
1,2,สั่งไป2 เมนู คือมัชฉะลาเต้ร้อน กับ ไอศครีมชาเข...,"[สั่ง, ไป, 2, เมนู, คือ, มัชฉะลา, เต้ร้อน, กับ..."
2,3,ครัววงเดือน \n\nหิวดึกๆ ตระเวนหาร้านทาน มาเจอ...,"[ครัววงเดือน, \n\n, หิว, ดึก, ๆ, ตระเวน, หา, ร..."
3,4,จะว่าเป็นเจ้าประจำก็คงไม่ผิด แต่ก็ไม่กล้า...,"[จะ, ว่า, เป็น, เจ้า, ประจำ, ก็, คง, ไม่, ผิด,..."
4,5,ถ้าคิดถึงสลัดผมคิดถึงร้านนี้เป็นร้านแรกๆเลยครั...,"[ถ้า, คิด, ถึง, สลัด, ผม, คิด, ถึง, ร้าน, นี้,..."


# TF-IDF

In [74]:
def tokenize(word_list):
    return word_list

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=tokenize, lowercase=False)
trainTfIdf = vectorizer.fit_transform(tr["tokenizedReview"])
testTfIdf = vectorizer.transform(ts["tokenizedReview"])

In [76]:
trainTfIdf.shape

(40000, 174166)

In [77]:
testTfIdf.shape

(6203, 174166)

# SVD 

In [78]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, n_iter=10, random_state=42)
trainFeatures = svd.fit_transform(trainTfIdf)
testFeatures = svd.transform(testTfIdf)

In [79]:
for i in range(300):
    tr["feature"+str(i)] = trainFeatures[:,i]
    ts["feature"+str(i)] = testFeatures[:,i]

In [80]:
tr.drop("tokenizedReview", axis=1, inplace=True)
ts.drop("tokenizedReview", axis=1, inplace=True)

In [81]:
tr.head()

,review,label,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,0.228316,-0.020764,-0.001777,0.100460,-0.017000,-0.035074,0.036856,0.015120,...,-0.001877,0.045393,0.010673,-0.024315,0.011204,0.028232,-0.036892,-0.003593,-0.005051,-0.000360
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,0.282241,-0.030783,-0.015046,0.039718,0.056262,-0.048687,-0.045608,0.015437,...,-0.016354,-0.020544,0.004761,0.009503,0.004973,-0.017164,-0.025104,-0.023011,0.012646,0.008128
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,0.286438,-0.032563,0.077864,0.004659,-0.024402,-0.038413,0.025931,0.006893,...,-0.001739,-0.000830,0.019939,-0.001312,-0.004875,-0.032898,0.033292,-0.002587,0.010849,0.020666
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,0.305668,-0.038370,0.133797,-0.122579,-0.032065,0.056910,0.039012,0.006311,...,-0.006993,-0.014081,-0.005929,0.036032,0.038386,-0.010596,-0.009205,0.002710,-0.004149,0.023743
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,0.236577,-0.020143,0.056803,-0.028959,-0.089624,0.023258,0.090730,0.012758,...,0.034450,0.000526,0.023553,0.022396,0.017152,-0.013019,-0.015032,0.016329,-0.014475,0.021125


In [82]:
ts.head()

,reviewID,review,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299
0,1,ร้านนี้จะอยู่เส้นสันกำแพง-แม่ออน เลยแยกบ่...,0.360498,0.581334,0.014321,-0.005751,-0.010322,-0.031789,-0.019707,0.036583,...,-0.003845,-0.018055,-0.019899,-0.015334,-0.017138,0.001517,-0.011955,0.015323,-0.004450,-0.005399
1,2,สั่งไป2 เมนู คือมัชฉะลาเต้ร้อน กับ ไอศครีมชาเข...,0.151875,-0.014623,-0.009261,0.030889,-0.034241,-0.017381,0.031627,0.025571,...,-0.008337,0.010086,-0.019927,0.028766,-0.002063,0.022431,0.026762,0.004502,0.004664,0.007936
2,3,ครัววงเดือน \n\nหิวดึกๆ ตระเวนหาร้านทาน มาเจอ...,0.313768,-0.025867,0.121377,0.146950,0.185460,0.069842,0.180548,-0.000253,...,0.009490,-0.039535,0.015618,-0.003521,0.002281,0.017251,0.034164,0.022599,0.032634,-0.019891
3,4,จะว่าเป็นเจ้าประจำก็คงไม่ผิด แต่ก็ไม่กล้า...,0.504221,-0.056977,0.123702,-0.008073,0.014924,-0.017910,0.086983,0.015646,...,-0.019678,-0.001157,0.007564,-0.021686,-0.028359,-0.011787,-0.006296,-0.033636,-0.039594,-0.025813
4,5,ถ้าคิดถึงสลัดผมคิดถึงร้านนี้เป็นร้านแรกๆเลยครั...,0.349388,-0.045990,0.209231,-0.088705,-0.087864,0.111806,0.017050,0.014967,...,-0.022208,-0.005811,0.006725,-0.004669,-0.021049,0.012265,-0.017928,0.001451,0.013642,-0.008648


In [83]:
tr.to_csv('../data/train_with_feature.csv', index=False)
ts.to_csv('../data/test_with_feature.csv', index=False)